# Introduction


In [1]:
import numpy as np
import pandas as pd
import pandapower as pp
import power_grid_model as pgm
from generate_fictional_dataset import generate_fictional_grid

In [2]:
# fictional grid parameters

n_node_per_feeder = 3
n_feeder_small = 100
n_feeder_big = 2

cable_length_km_min = 0.8
cable_length_km_max = 1.2
load_p_w_max = 0.4e6 * 0.8 * 10
load_p_w_min = 0.4e6 * 1.2 * 10
pf = 0.95


# Big Network

In [3]:
fictional_dataset = generate_fictional_grid(
    n_node_per_feeder=n_node_per_feeder,
    n_feeder=n_feeder_big,
    cable_length_km_min=cable_length_km_min,
    cable_length_km_max=cable_length_km_max,
    load_p_w_max=load_p_w_max,
    load_p_w_min=load_p_w_min,
    pf=pf
)

pp_net = fictional_dataset['pp_net']
pgm_dataset = fictional_dataset['pgm_dataset']
pgm_model = pgm.PowerGridModel(pgm_dataset)

In [4]:
pgm_result = pgm_model.calculate_power_flow(error_tolerance=1e-8)

In [5]:
pd.DataFrame(pgm_result['node'])

,id,energized,u_pu,u,u_angle
0,0,1,1.045099,10450.985652,-0.010124
1,1,1,1.034048,10340.476391,-0.019021
2,2,1,1.025966,10259.655412,-0.025654
3,3,1,1.022055,10220.549528,-0.028898
4,4,1,1.034118,10341.182217,-0.018987
5,5,1,1.027605,10276.050011,-0.024337
6,6,1,1.023892,10238.919244,-0.027414


In [19]:
pp.runpp(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)

In [20]:
np.abs(pp_net.res_bus['vm_pu'] - pgm_result['node']['u_pu']).max()

0.004991596381541008

In [21]:
pp_net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,11.587582,3.952239,-11.500224,-3.823394,0.087358,0.128845,0.673193,0.673440,0.673440,1.050000,0.000000,1.038992,-0.505517,67.343969
1,7.919592,2.646498,-7.875374,-2.588824,0.044218,0.057674,0.464000,0.464256,0.464256,1.038992,-0.505517,1.030942,-0.882278,46.425640
2,3.980316,1.308580,-3.969471,-1.304702,0.010845,0.003878,0.234644,0.234887,0.234887,1.030942,-0.882278,1.027047,-1.066553,23.488662
3,11.548925,3.922521,-11.462359,-3.794949,0.086566,0.127572,0.670654,0.670900,0.670900,1.050000,0.000000,1.039062,-0.503576,67.089969
4,7.159376,2.380627,-7.127156,-2.341022,0.032220,0.039606,0.419224,0.419452,0.419452,1.039062,-0.503576,1.032575,-0.807487,41.945215
5,3.724711,1.222692,-3.715089,-1.221091,0.009622,0.001601,0.219196,0.219443,0.219443,1.032575,-0.807487,1.028876,-0.982243,21.944313


In [14]:
np.abs(pp_net.res_line['loading_percent'] * 1e-2  - pgm_result['line']['loading'])

0    0.002588
1    0.001796
2    0.000911
3    0.002568
4    0.001616
5    0.000848
Name: loading_percent, dtype: float64

In [9]:
pp_net.res_line['p_from_mw']

0    11.587582
1     7.919592
2     3.980316
3    11.548925
4     7.159376
5     3.724711
Name: p_from_mw, dtype: float64

In [10]:
pgm_result['line']['loading']

array([0.67602784, 0.46605214, 0.23579798, 0.67346784, 0.42106863,
       0.22029102])